<a href="https://colab.research.google.com/github/vasudevgupta7/bigbird/blob/main/notebooks/evaluate_nq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue May  4 07:08:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
%%capture
!git clone https://github.com/vasudevgupta7/bigbird
!cd bigbird/natural-questions && pip3 install -r requirements.txt

In [6]:
cd bigbird/natural-questions

/content/bigbird/natural-questions


In [7]:
!mkdir natural-questions-validation
!wget https://huggingface.co/datasets/vasudevgupta/natural-questions-validation/resolve/main/natural_questions-validation.arrow -P natural-questions-validation
!wget https://huggingface.co/datasets/vasudevgupta/natural-questions-validation/resolve/main/dataset_info.json -P natural-questions-validation
!wget https://huggingface.co/datasets/vasudevgupta/natural-questions-validation/resolve/main/state.json -P natural-questions-validation

--2021-05-04 18:17:41--  https://huggingface.co/datasets/vasudevgupta/natural-questions-validation/resolve/main/natural_questions-validation.arrow
Resolving huggingface.co (huggingface.co)... 34.201.172.85
Connecting to huggingface.co (huggingface.co)|34.201.172.85|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/vasudevgupta/natural-questions-validation/80201e1a5434183f43f73f1b7f57dfe9ea6c50da4f45007aeb192a2ec6556e70 [following]
--2021-05-04 18:17:42--  https://cdn-lfs.huggingface.co/datasets/vasudevgupta/natural-questions-validation/80201e1a5434183f43f73f1b7f57dfe9ea6c50da4f45007aeb192a2ec6556e70
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 13.35.24.38, 13.35.24.76, 13.35.24.87, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|13.35.24.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2353975312 (2.2G) [application/octet-stream]
Saving to: ‘natural-questio

In [8]:
from datasets import load_dataset, load_from_disk

dataset = load_from_disk("natural-questions-validation")
dataset

Dataset({
    features: ['annotations', 'document', 'id', 'question'],
    num_rows: 7830
})

In [9]:
def format_dataset(sample):
  question = sample['question']['text']
  context = sample['document']['tokens']['token']
  is_html = sample['document']['tokens']['is_html']
  long_answers = sample['annotations']['long_answer']
  short_answers = sample['annotations']['short_answers']

  context_string =  " ".join([context[i] for i in range(len(context)) if not is_html[i]])

  # 0 - No ; 1 - Yes
  for answer in sample['annotations']['yes_no_answer']:
    if answer == 0 or answer == 1:
      return {"question": question, "context": context_string, "short": [], "long": [], "category": "no" if answer == 0 else "yes"}

  short_targets = []
  for s in short_answers:
    short_targets.extend(s['text'])

  long_targets = []
  for s in long_answers:
    if s['start_token'] == -1:
      continue
    answer = context[s['start_token']: s['end_token']]
    html = is_html[s['start_token']: s['end_token']]
    new_answer = " ".join([answer[i] for i in range(len(answer)) if not html[i]])
    if new_answer not in long_targets:
      long_targets.append(new_answer)
  return {"question": question, "context": context_string, "short": short_targets, "long": long_targets, "category": "long_short"}

In [10]:
# dataset = dataset.select(range(10))
cleaned_dataset = dataset.map(format_dataset).remove_columns(["annotations", "document", "id"])
cleaned_dataset

Dataset({
    features: ['category', 'context', 'long', 'question', 'short'],
    num_rows: 7830
})

In [11]:
short_validation_dataset = cleaned_dataset.filter(lambda x: (len(x['question']) + len(x['context'])) < 4 * 4096)
short_validation_dataset

Dataset({
    features: ['category', 'context', 'long', 'question', 'short'],
    num_rows: 2721
})

In [ ]:
PUNCTUATION_SET_TO_EXCLUDE = set(''.join(['‘', '’', '´', '`', '.', ',', '-', '"']))

def get_sub_answers(answers, begin=0, end=None):
  return [" ".join(x.split(" ")[begin:end]) for x in answers if len(x.split(" ")) > 1]

def expand_to_aliases(given_answers, make_sub_answers=False):
  if make_sub_answers:
    # if answers are longer than one word, make sure a predictions is correct if it coresponds to the complete 1: or :-1 sub word
    # *e.g.* if the correct answer contains a prefix such as "the", or "a"
    given_answers = given_answers + get_sub_answers(given_answers, begin=1) + get_sub_answers(given_answers, end=-1)
  answers = []
  for answer in given_answers:
    alias = answer.replace('_', ' ').lower()
    alias = ''.join(c if c not in PUNCTUATION_SET_TO_EXCLUDE else ' ' for c in alias)
    answers.append(' '.join(alias.split()).strip())
  return set(answers)

In [ ]:
def get_best_valid_start_end_idx(start_scores, end_scores, top_k=1, max_size=100):
    best_start_scores, best_start_idx = torch.topk(start_scores, top_k)
    best_end_scores, best_end_idx = torch.topk(end_scores, top_k)

    widths = best_end_idx[:, None] - best_start_idx[None, :]
    mask = torch.logical_or(widths < 0, widths > max_size)
    scores = (best_end_scores[:, None] + best_start_scores[None, :]) - (1e8 * mask)
    best_score = torch.argmax(scores).item()

    return best_start_idx[best_score % top_k], best_end_idx[best_score // top_k]

In [ ]:
def evaluate(example):

    with torch.no_grad():
        start_scores, end_scores = model(input_ids=input_ids).to_tuple()

    start_score, end_score = get_best_valid_start_end_idx(start_scores[0], end_scores[0], top_k=8, max_size=16)

    # Let's convert the input ids back to actual tokens 
    all_tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0].tolist())
    answer_tokens = all_tokens[start_score: end_score + 1]

    example["output"] = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))
    #.replace('"', '')  # remove space prepending space token and remove unnecessary '"'

    answers = expand_to_aliases(example["targets"], make_sub_answers=True)
    predictions = expand_to_aliases([example["output"]])

    # if there is a common element, it's a match
    example["match"] = len(list(answers & predictions)) > 0

    return example


In [ ]:
def evaluate(example, print_output=False):
    # encode question and context so that they are seperated by a tokenizer.sep_token and cut at max_length
    inputs = tokenizer(example["question"], example["context"], return_tensors="pt", max_length=4096, padding="max_length", truncation=True)
    inputs = {k: inputs[k].to(DEVICE) for k in inputs}

    with torch.no_grad():
        outputs = model(input_ids=input_ids)
        _, start = outputs["start_logits"].max(dim=-1)
        _, end = outputs["end_logits"].max(dim=-1)
        start, end = start.item(), end.item()
        _, category = outputs["cls_out"].max(dim=-1)

    predicted_category = CATEGORY_MAPPING[category.item()]
    original_category = CATEGORY_MAPPING[example['category']]

    input_ids = input_ids[0].cpu().tolist()
    answer = tokenizer.decode(input_ids[start: end+1])
    orig_answer = tokenizer.decode(input_ids[example["start_token"]: example["end_token"]+1])

    if print_output:
        print(tokenizer.decode(input_ids[1:input_ids.index(tokenizer.sep_token_id)]))
        print("original_category:", original_category)
        print("predicted_category:", predicted_category)
        if original_category in ['long', 'short']:
            print("original_answer:", orig_answer)
            print("predicted_answer:", answer, end="\n\n")

    example["orig_answer"] = orig_answer
    example["answer"] = answer
    if predicted_category in ['yes', 'no', 'null']:
        example['match'] = 1 if predicted_category == original_category else 0
    else:
        example['match'] = 1 if answer == orig_answer else 0 # exact match
    return example

In [ ]:
MODEL_ID = "vasudevgupta/bigbird-roberta-natural-questions"
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
revision = "4daa96c5befa9b728c47a77280ef9484377d7791"

model = BigBirdForNaturalQuestions.from_pretrained(MODEL_ID, revision=revision).to(DEVICE)
tokenizer = BigBirdTokenizer.from_pretrained(MODEL_ID, revision=revision)

val_dataset = load_dataset("json", data_files="nq-validation.jsonl")['train']
# val_dataset = val_dataset.filter(lambda x: x['category'] != 0)
print(val_dataset)

Using custom data configuration default-64367e4e8e6b551b
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-64367e4e8e6b551b/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02)


Dataset({
    features: ['input_ids', 'start_token', 'end_token', 'category'],
    num_rows: 10662
})


In [ ]:
short_validation_dataset = short_validation_dataset.map(evaluate)

In [ ]:
total = len(short_validation_dataset)
matched = len(short_validation_dataset.filter(lambda x: x["match"] == 1))
print("EM score:", (matched / total)*100, "%")


EM score: 25.447830101569714 %


In [ ]:
# short_validation_dataset.save_to_disk("predictions-nq")